## PLS
eerste grote stappen in de pls

In [10]:
import pandas as pd
from sklearn.cross_decomposition import PLSRegression
import calendar

#loading dataframes 

File_url_d = 'https://raw.githubusercontent.com/Sebasleen/Seminargroup/Seminar/managed_portfolios_anom_d_50.csv'

r_daily = pd.read_csv(File_url_d)

# set date to datetime format and set the date to the index 

r_daily['date'] = pd.to_datetime(r_daily['date'])
r_daily.set_index('date', inplace=True)

# missing list = ['dur', 'divg', 'invaci', 'ipo'] -> deze worden gemaakt uit daily stock data doormiddel van een signals list?

# following the procedure in the paper, if there are observations missing we set them to 0. 

r_daily.fillna(0, inplace=True)





In [11]:
import pandas as pd
from sklearn.cross_decomposition import PLSRegression
import calendar

# Define initial parameters
initial_start_date = pd.to_datetime('1967-07-01')
initial_training_years = 10

# Define factors
factors = [
    'rme', 're_ew', 'r_size', 'r_value', 'r_prof', 'r_valprof', 'r_fscore',
    'r_debtiss', 'r_repurch', 'r_nissa', 'r_accruals', 'r_growth', 'r_aturnover',
    'r_gmargins', 'r_divp', 'r_ep', 'r_cfp', 'r_noa', 'r_inv', 'r_invcap',
    'r_igrowth', 'r_sgrowth', 'r_lev', 'r_roaa', 'r_roea', 'r_sp', 'r_gltnoa',
    'r_indmom', 'r_valmom', 'r_valmomprof', 'r_shortint', 'r_mom12', 'r_momrev',
    'r_lrrev', 'r_valuem', 'r_nissm', 'r_sue', 'r_roe', 'r_rome', 'r_roa',
    'r_strev', 'r_ivol', 'r_betaarb', 'r_season', 'r_indrrev', 'r_indrrevlv',
    'r_indmomrev', 'r_ciss', 'r_price', 'r_age', 'r_shvol'
]

# Create an empty dataframe for monthly PLS returns
pls_return_df = pd.DataFrame(columns=['date'] + [f"PLS{i+1}" for i in range(len(factors))])

# Initialize an empty list to store PLS weights
pls_weights_list = []

# Loop through years and months
for year in range(1973, 1975):
    for month in range(6, 13) if year == 1973 else range(1, 13):

        # Define start and end dates
        last_day = calendar.monthrange(year, month)[1]
        end_date = pd.to_datetime(f'{year}-{month}-{last_day}')

        # For the expanding window, start date becomes initial start date
        start_date = initial_start_date

        # Select data range for PLS analysis
        pls_data = r_daily.loc[start_date:end_date]

        # Separate features (X) and target (y)
        X = pls_data[factors]
        y = pls_data['r_mom12']

        # Fit PLS model
        pls = PLSRegression(n_components=10)  # Adjust number of components as needed
        pls.fit(X, y)

        # Calculate predicted values
        predictions = pls.predict(X)

        # Extract and store PLS weights
        pls_weights = pd.DataFrame(pls.x_weights_, index=factors, columns=[f"PLS{i+1}" for i in range(pls.x_weights_.shape[1])])
        pls_weights_list.append(pls_weights)

        # Append the monthly PLS returns to the dataframe
        pls_return_data = {'date': end_date}
        for f in range(len(factors)):
            pls_return_data[f"PLS{f+1}"] = predictions[f]
        pls_return_df = pls_return_df.append(pls_return_data, ignore_index=True)

# Print the resulting dataframe
print("PLS Returns:")
print(pls_return_df)

# Print PLS weights
for i, weights in enumerate(pls_weights_list, start=1):
    print(f"PLS Weights for month {i}:")
    print(weights)


AttributeError: 'DataFrame' object has no attribute 'append'